In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('UNSW_NB15_train-set.csv').drop(['id'], axis=1)


In [3]:
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers
from keras.wrappers.scikit_learn import KerasClassifier


In [4]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import RobustScaler

In [5]:
# separate categroicals with non categoricals 
output1 = train_df[['attack_cat']].copy()  # multiclass
output2 = train_df[['label']].copy()       # binary

input_categroicals = train_df.dtypes[train_df.dtypes == object].index.tolist()[:-1]
input_numericals = train_df.dtypes[train_df.dtypes != object].index.tolist()[:-1]

input_categorical_values = train_df[input_categroicals].copy()
input_numerical_values = train_df[input_numericals].copy()

In [6]:
# encoding the values
output1 = pd.get_dummies(output1)

rbs= RobustScaler()
rbs.fit(input_numerical_values)
input_numerical_values = pd.DataFrame(rbs.transform(input_numerical_values), columns=input_numericals)
input_numerical_values

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,0.179469,0.4,0.2,-0.131902,0.007260,-0.025220,-0.010417,0.892857,-0.009738,0.253129,...,-0.166667,0.00,0.0,-0.181818,0.0,0.0,0.0,-0.285714,-0.3,0.0
1,0.970450,1.2,3.6,0.233129,37.976407,-0.025185,-1.000000,0.884921,-0.009803,18.032918,...,-0.166667,0.00,0.0,-0.090909,0.0,0.0,0.0,-0.285714,0.2,0.0
2,2.427244,0.6,1.4,-0.050613,11.816697,-0.025700,-1.000000,0.884921,-0.009880,2.136200,...,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,-0.142857,0.2,0.0
3,2.514830,1.0,1.0,0.151840,0.549909,-0.025704,-1.000000,0.884921,-0.009867,0.068652,...,0.000000,0.00,0.0,0.000000,1.0,1.0,0.0,-0.142857,-0.3,0.0
4,0.670406,0.8,0.4,0.079755,0.094374,-0.025546,0.000000,0.884921,-0.009801,0.091221,...,0.000000,0.25,0.0,3.363636,0.0,0.0,0.0,-0.142857,3.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,-0.002355,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,...,3.666667,5.75,6.0,1.909091,0.0,0.0,0.0,3.000000,2.0,0.0
175337,0.754692,0.8,0.6,0.145706,0.172414,-0.025544,0.000000,0.884921,-0.009798,0.124133,...,-0.166667,0.00,0.0,-0.090909,0.0,0.0,0.0,-0.285714,-0.3,0.0
175338,-0.002355,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,...,0.166667,0.50,1.0,0.909091,0.0,0.0,0.0,0.000000,0.8,0.0
175339,-0.002355,0.0,-0.2,-0.242331,-0.148820,0.863309,0.000000,-0.115079,0.560186,-0.051967,...,4.666667,7.25,6.5,2.454545,0.0,0.0,0.0,3.857143,2.6,0.0


In [7]:
(train_df['attack_cat'].value_counts() / train_df.shape[0]).round(4) * 100

Normal            31.94
Generic           22.81
Exploits          19.04
Fuzzers           10.37
DoS                6.99
Reconnaissance     5.98
Analysis           1.14
Backdoor           1.00
Shellcode          0.65
Worms              0.07
Name: attack_cat, dtype: float64

In [8]:
output1

,attack_cat_Analysis,attack_cat_Backdoor,attack_cat_DoS,attack_cat_Exploits,attack_cat_Fuzzers,attack_cat_Generic,attack_cat_Normal,attack_cat_Reconnaissance,attack_cat_Shellcode,attack_cat_Worms
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
175336,0,0,0,0,0,1,0,0,0,0
175337,0,0,0,0,0,0,0,0,1,0
175338,0,0,0,0,0,1,0,0,0,0
175339,0,0,0,0,0,1,0,0,0,0


In [9]:
 from sklearn.metrics import f1_score


In [11]:
import keras.backend as K

In [12]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2*(precision*recall)/(precision+recall+K.epsilon())
    

In [13]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 39)                1560      
                                                                 
 dense_1 (Dense)             (None, 10)                400       
                                                                 
Total params: 1,960
Trainable params: 1,960
Non-trainable params: 0
_________________________________________________________________


In [14]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 8s 1ms/step - loss: 46.7196 - accuracy: 0.7119 - precision: 0.8595 - recall: 0.5700 - get_f1: 0.6808
Epoch 2/10
5480/5480 [==============================] - 7s 1ms/step - loss: 32.8047 - accuracy: 0.7547 - precision: 0.8668 - recall: 0.6443 - get_f1: 0.7371
Epoch 3/10
5480/5480 [==============================] - 8s 1ms/step - loss: 26.9324 - accuracy: 0.7659 - precision: 0.8727 - recall: 0.6663 - get_f1: 0.7538
Epoch 4/10
5480/5480 [==============================] - 8s 2ms/step - loss: 23.2289 - accuracy: 0.7730 - precision: 0.8757 - recall: 0.6774 - get_f1: 0.7622
Epoch 5/10
5480/5480 [==============================] - 7s 1ms/step - loss: 16.5810 - accuracy: 0.7796 - precision: 0.8820 - recall: 0.6855 - get_f1: 0.7697
Epoch 6/10
5480/5480 [==============================] - 7s 1ms/step - loss: 11.2962 - accuracy: 0.7793 - precision: 0.8794 - recall: 0.6867 - get_f1: 0.7695
Epoch 7/10
5480/5480 [==============================] - 7s

In [15]:
model = Sequential()
model.add(Dense(26, input_dim=39, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 26)                1040      
                                                                 
 dense_3 (Dense)             (None, 10)                270       
                                                                 
Total params: 1,310
Trainable params: 1,310
Non-trainable params: 0
_________________________________________________________________


In [16]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 8s 1ms/step - loss: 31.2139 - accuracy: 0.6997 - precision: 0.8572 - recall: 0.5586 - get_f1: 0.6722
Epoch 2/10
5480/5480 [==============================] - 8s 1ms/step - loss: 29.9355 - accuracy: 0.7406 - precision: 0.8652 - recall: 0.6260 - get_f1: 0.7242
Epoch 3/10
5480/5480 [==============================] - 7s 1ms/step - loss: 24.7727 - accuracy: 0.7529 - precision: 0.8709 - recall: 0.6491 - get_f1: 0.7419
Epoch 4/10
5480/5480 [==============================] - 7s 1ms/step - loss: 20.7897 - accuracy: 0.7645 - precision: 0.8760 - recall: 0.6638 - get_f1: 0.7534
Epoch 5/10
5480/5480 [==============================] - 7s 1ms/step - loss: 18.5090 - accuracy: 0.7727 - precision: 0.8751 - recall: 0.6785 - get_f1: 0.7627
Epoch 6/10
5480/5480 [==============================] - 7s 1ms/step - loss: 21.1932 - accuracy: 0.7746 - precision: 0.8760 - recall: 0.6836 - get_f1: 0.7662
Epoch 7/10
5480/5480 [==============================] - 7s

In [17]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(26, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 39)                1560      
                                                                 
 dense_5 (Dense)             (None, 26)                1040      
                                                                 
 dense_6 (Dense)             (None, 10)                270       
                                                                 
Total params: 2,870
Trainable params: 2,870
Non-trainable params: 0
_________________________________________________________________


In [18]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 8s 1ms/step - loss: 46.8640 - accuracy: 0.7103 - precision: 0.8634 - recall: 0.5641 - get_f1: 0.6764
Epoch 2/10
5480/5480 [==============================] - 8s 1ms/step - loss: 7.0685 - accuracy: 0.7690 - precision: 0.8909 - recall: 0.6464 - get_f1: 0.7470
Epoch 3/10
5480/5480 [==============================] - 7s 1ms/step - loss: 0.6123 - accuracy: 0.7721 - precision: 0.8930 - recall: 0.6578 - get_f1: 0.7554
Epoch 4/10
5480/5480 [==============================] - 7s 1ms/step - loss: 0.5843 - accuracy: 0.7703 - precision: 0.8921 - recall: 0.6583 - get_f1: 0.7555
Epoch 5/10
5480/5480 [==============================] - 7s 1ms/step - loss: 0.5690 - accuracy: 0.7726 - precision: 0.8906 - recall: 0.6644 - get_f1: 0.7589
Epoch 6/10
5480/5480 [==============================] - 7s 1ms/step - loss: 0.5572 - accuracy: 0.7759 - precision: 0.8940 - recall: 0.6683 - get_f1: 0.7628
Epoch 7/10
5480/5480 [==============================] - 7s 1ms/

In [19]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(26, activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 39)                1560      
                                                                 
 dense_8 (Dense)             (None, 26)                1040      
                                                                 
 dense_9 (Dense)             (None, 17)                459       
                                                                 
 dense_10 (Dense)            (None, 10)                180       
                                                                 
Total params: 3,239
Trainable params: 3,239
Non-trainable params: 0
_________________________________________________________________


In [20]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 9s 1ms/step - loss: 9.1279 - accuracy: 0.6999 - precision: 0.8959 - recall: 0.5442 - get_f1: 0.6710
Epoch 2/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6453 - accuracy: 0.7541 - precision: 0.8897 - recall: 0.6333 - get_f1: 0.7375
Epoch 3/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6225 - accuracy: 0.7654 - precision: 0.8873 - recall: 0.6549 - get_f1: 0.7515
Epoch 4/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5770 - accuracy: 0.7717 - precision: 0.8911 - recall: 0.6642 - get_f1: 0.7591
Epoch 5/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5621 - accuracy: 0.7759 - precision: 0.8922 - recall: 0.6693 - get_f1: 0.7629
Epoch 6/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5515 - accuracy: 0.7781 - precision: 0.8933 - recall: 0.6720 - get_f1: 0.7650
Epoch 7/10
5480/5480 [==============================] - 8s 1ms/s

In [21]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(17, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 39)                1560      
                                                                 
 dense_12 (Dense)            (None, 17)                680       
                                                                 
 dense_13 (Dense)            (None, 11)                198       
                                                                 
 dense_14 (Dense)            (None, 10)                120       
                                                                 
Total params: 2,558
Trainable params: 2,558
Non-trainable params: 0
_________________________________________________________________


In [22]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 8s 1ms/step - loss: 8.3712 - accuracy: 0.7228 - precision: 0.8806 - recall: 0.5799 - get_f1: 0.6945
Epoch 2/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6260 - accuracy: 0.7583 - precision: 0.8880 - recall: 0.6389 - get_f1: 0.7410
Epoch 3/10
5480/5480 [==============================] - 7s 1ms/step - loss: 0.6088 - accuracy: 0.7652 - precision: 0.8929 - recall: 0.6484 - get_f1: 0.7492
Epoch 4/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5790 - accuracy: 0.7694 - precision: 0.8924 - recall: 0.6564 - get_f1: 0.7543
Epoch 5/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5665 - accuracy: 0.7740 - precision: 0.8928 - recall: 0.6625 - get_f1: 0.7584
Epoch 6/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5615 - accuracy: 0.7768 - precision: 0.8952 - recall: 0.6665 - get_f1: 0.7620
Epoch 7/10
5480/5480 [==============================] - 8s 1ms/s

In [23]:
model = Sequential()
model.add(Dense(39, input_dim=39, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 39)                1560      
                                                                 
 dense_16 (Dense)            (None, 11)                440       
                                                                 
 dense_17 (Dense)            (None, 7)                 84        
                                                                 
 dense_18 (Dense)            (None, 10)                80        
                                                                 
Total params: 2,164
Trainable params: 2,164
Non-trainable params: 0
_________________________________________________________________


In [24]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 8s 1ms/step - loss: 6.3788 - accuracy: 0.7078 - precision: 0.8937 - recall: 0.5385 - get_f1: 0.6642
Epoch 2/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6487 - accuracy: 0.7556 - precision: 0.8939 - recall: 0.6283 - get_f1: 0.7356
Epoch 3/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6117 - accuracy: 0.7632 - precision: 0.8916 - recall: 0.6440 - get_f1: 0.7458
Epoch 4/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5951 - accuracy: 0.7676 - precision: 0.8893 - recall: 0.6563 - get_f1: 0.7531
Epoch 5/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6238 - accuracy: 0.7693 - precision: 0.8896 - recall: 0.6613 - get_f1: 0.7567
Epoch 6/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.5746 - accuracy: 0.7722 - precision: 0.8943 - recall: 0.6628 - get_f1: 0.7593
Epoch 7/10
5480/5480 [==============================] - 8s 1ms/s

In [25]:
model = Sequential()
model.add(Dense(17, input_dim=39, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 17)                680       
                                                                 
 dense_20 (Dense)            (None, 11)                198       
                                                                 
 dense_21 (Dense)            (None, 7)                 84        
                                                                 
 dense_22 (Dense)            (None, 10)                80        
                                                                 
Total params: 1,042
Trainable params: 1,042
Non-trainable params: 0
_________________________________________________________________


In [26]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 10s 2ms/step - loss: 12.3298 - accuracy: 0.6702 - precision: 0.8806 - recall: 0.5060 - get_f1: 0.6364
Epoch 2/10
5480/5480 [==============================] - 9s 2ms/step - loss: 0.7810 - accuracy: 0.7332 - precision: 0.9033 - recall: 0.5675 - get_f1: 0.6940
Epoch 3/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.7534 - accuracy: 0.7479 - precision: 0.8802 - recall: 0.6133 - get_f1: 0.7202
Epoch 4/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6601 - accuracy: 0.7528 - precision: 0.8795 - recall: 0.6368 - get_f1: 0.7365
Epoch 5/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6318 - accuracy: 0.7526 - precision: 0.8865 - recall: 0.6309 - get_f1: 0.7349
Epoch 6/10
5480/5480 [==============================] - 8s 1ms/step - loss: 0.6137 - accuracy: 0.7616 - precision: 0.8852 - recall: 0.6387 - get_f1: 0.7398
Epoch 7/10
5480/5480 [==============================] - 8s 2ms

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

data = input_numerical_values.values
data = data.reshape(input_numerical_values.shape[0], 1, 39, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 37, 32)         128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 18, 32)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 576)               0         
                                                                 
 dense_23 (Dense)            (None, 128)               73856     
                                                                 
 dense_24 (Dense)            (None, 10)                1290      
                                                                 
Total params: 75,274
Trainable params: 75,274
Non-trainable params: 0
__________________________________________________

In [28]:
model.fit(data, output1, epochs=10, batch_size=32)

Epoch 1/10
5480/5480 [==============================] - 14s 2ms/step - loss: 50.7773 - accuracy: 0.7466 - precision: 0.8455 - recall: 0.6550 - get_f1: 0.7352
Epoch 2/10
5480/5480 [==============================] - 13s 2ms/step - loss: 0.6842 - accuracy: 0.7810 - precision: 0.8720 - recall: 0.7012 - get_f1: 0.7758
Epoch 3/10
5480/5480 [==============================] - 13s 2ms/step - loss: 0.6726 - accuracy: 0.7898 - precision: 0.8826 - recall: 0.7067 - get_f1: 0.7833
Epoch 4/10
5480/5480 [==============================] - 13s 2ms/step - loss: 0.6020 - accuracy: 0.7943 - precision: 0.8906 - recall: 0.7072 - get_f1: 0.7868
Epoch 5/10
5480/5480 [==============================] - 13s 2ms/step - loss: 0.5358 - accuracy: 0.7977 - precision: 0.8896 - recall: 0.7137 - get_f1: 0.7904
Epoch 6/10
5480/5480 [==============================] - 14s 3ms/step - loss: 0.5183 - accuracy: 0.8004 - precision: 0.8920 - recall: 0.7161 - get_f1: 0.7928
Epoch 7/10
5480/5480 [==============================] - 1

In [29]:
model = Sequential()
model.add(Conv2D(17, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 1, 37, 17)         68        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 18, 17)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 306)               0         
                                                                 
 dense_25 (Dense)            (None, 64)                19648     
                                                                 
 dense_26 (Dense)            (None, 10)                650       
                                                                 
Total params: 20,366
Trainable params: 20,366
Non-trainable params: 0
__________________________________________________

In [30]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 12s 2ms/step - loss: 16.7334 - accuracy: 0.7483 - precision: 0.8601 - recall: 0.6474 - get_f1: 0.7348
Epoch 2/10
5480/5480 [==============================] - 11s 2ms/step - loss: 2.1474 - accuracy: 0.7821 - precision: 0.8754 - recall: 0.6969 - get_f1: 0.7744
Epoch 3/10
5480/5480 [==============================] - 11s 2ms/step - loss: 1.2538 - accuracy: 0.7882 - precision: 0.8813 - recall: 0.7015 - get_f1: 0.7796
Epoch 4/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.5398 - accuracy: 0.7939 - precision: 0.8881 - recall: 0.7085 - get_f1: 0.7867
Epoch 5/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.5116 - accuracy: 0.7985 - precision: 0.8916 - recall: 0.7134 - get_f1: 0.7910
Epoch 6/10
5480/5480 [==============================] - 11s 2ms/step - loss: 0.5036 - accuracy: 0.8013 - precision: 0.8919 - recall: 0.7178 - get_f1: 0.7939
Epoch 7/10
5480/5480 [==============================] - 1

In [31]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 1, 37, 42)         168       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 18, 42)        0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 756)               0         
                                                                 
 dense_27 (Dense)            (None, 150)               113550    
                                                                 
 dense_28 (Dense)            (None, 10)                1510      
                                                                 
Total params: 115,228
Trainable params: 115,228
Non-trainable params: 0
________________________________________________

In [32]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 16s 3ms/step - loss: 30.3787 - accuracy: 0.7494 - precision: 0.8415 - recall: 0.6677 - get_f1: 0.7420
Epoch 2/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5590 - accuracy: 0.7862 - precision: 0.8722 - recall: 0.7080 - get_f1: 0.7801
Epoch 3/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5513 - accuracy: 0.7920 - precision: 0.8791 - recall: 0.7165 - get_f1: 0.7879
Epoch 4/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5221 - accuracy: 0.7965 - precision: 0.8854 - recall: 0.7170 - get_f1: 0.7907
Epoch 5/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5167 - accuracy: 0.8011 - precision: 0.8919 - recall: 0.7186 - get_f1: 0.7944
Epoch 6/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5042 - accuracy: 0.8035 - precision: 0.8947 - recall: 0.7198 - get_f1: 0.7962
Epoch 7/10
5480/5480 [==============================] - 1

In [33]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 1, 37, 42)         168       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 18, 42)        0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 756)               0         
                                                                 
 dense_29 (Dense)            (None, 150)               113550    
                                                                 
 dense_30 (Dense)            (None, 15)                2265      
                                                                 
 dense_31 (Dense)            (None, 10)                160       
                                                     

In [34]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 16s 3ms/step - loss: 1.0491 - accuracy: 0.7435 - precision: 0.8822 - recall: 0.6193 - get_f1: 0.7239
Epoch 2/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5963 - accuracy: 0.7620 - precision: 0.8810 - recall: 0.6571 - get_f1: 0.7508
Epoch 3/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5630 - accuracy: 0.7723 - precision: 0.8840 - recall: 0.6707 - get_f1: 0.7608
Epoch 4/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5510 - accuracy: 0.7773 - precision: 0.8876 - recall: 0.6760 - get_f1: 0.7655
Epoch 5/10
5480/5480 [==============================] - 15s 3ms/step - loss: 0.5496 - accuracy: 0.7804 - precision: 0.8896 - recall: 0.6818 - get_f1: 0.7701
Epoch 6/10
5480/5480 [==============================] - 16s 3ms/step - loss: 0.5281 - accuracy: 0.7837 - precision: 0.8942 - recall: 0.6839 - get_f1: 0.7731
Epoch 7/10
5480/5480 [==============================] - 16

In [35]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 1, 37, 42)         168       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 18, 42)        0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 756)               0         
                                                                 
 dense_32 (Dense)            (None, 10)                7570      
                                                                 
Total params: 7,738
Trainable params: 7,738
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 13s 2ms/step - loss: 40.6706 - accuracy: 0.7273 - precision: 0.8383 - recall: 0.6267 - get_f1: 0.7136
Epoch 2/10
5480/5480 [==============================] - 12s 2ms/step - loss: 20.7099 - accuracy: 0.7594 - precision: 0.8442 - recall: 0.6877 - get_f1: 0.7566
Epoch 3/10
5480/5480 [==============================] - 12s 2ms/step - loss: 7.9619 - accuracy: 0.7690 - precision: 0.8515 - recall: 0.7001 - get_f1: 0.7671
Epoch 4/10
5480/5480 [==============================] - 12s 2ms/step - loss: 1.0633 - accuracy: 0.7758 - precision: 0.8610 - recall: 0.7028 - get_f1: 0.7725
Epoch 5/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.6096 - accuracy: 0.7809 - precision: 0.8691 - recall: 0.7043 - get_f1: 0.7766
Epoch 6/10
5480/5480 [==============================] - 12s 2ms/step - loss: 0.5710 - accuracy: 0.7832 - precision: 0.8736 - recall: 0.7083 - get_f1: 0.7808
Epoch 7/10
5480/5480 [==============================] - 

In [37]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 3), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 1, 37, 42)         168       
                                                                 
 flatten_5 (Flatten)         (None, 1554)              0         
                                                                 
 dense_33 (Dense)            (None, 150)               233250    
                                                                 
 dense_34 (Dense)            (None, 10)                1510      
                                                                 
Total params: 234,928
Trainable params: 234,928
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 23s 4ms/step - loss: 16.9466 - accuracy: 0.7609 - precision: 0.8511 - recall: 0.6815 - get_f1: 0.7548
Epoch 2/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5343 - accuracy: 0.7905 - precision: 0.8796 - recall: 0.7127 - get_f1: 0.7859
Epoch 3/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.8080 - accuracy: 0.7942 - precision: 0.8835 - recall: 0.7139 - get_f1: 0.7882
Epoch 4/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5200 - accuracy: 0.7987 - precision: 0.8900 - recall: 0.7164 - get_f1: 0.7922
Epoch 5/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5368 - accuracy: 0.8013 - precision: 0.8933 - recall: 0.7178 - get_f1: 0.7944
Epoch 6/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.5766 - accuracy: 0.8038 - precision: 0.8918 - recall: 0.7230 - get_f1: 0.7970
Epoch 7/10
5480/5480 [==============================] - 2

In [39]:
model = Sequential()
model.add(Conv2D(42, kernel_size=(1, 5), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 1, 35, 42)         252       
                                                                 
 flatten_6 (Flatten)         (None, 1470)              0         
                                                                 
 dense_35 (Dense)            (None, 150)               220650    
                                                                 
 dense_36 (Dense)            (None, 10)                1510      
                                                                 
Total params: 222,412
Trainable params: 222,412
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 20s 4ms/step - loss: 23.7325 - accuracy: 0.7584 - precision: 0.8550 - recall: 0.6729 - get_f1: 0.7507
Epoch 2/10
5480/5480 [==============================] - 20s 4ms/step - loss: 0.6257 - accuracy: 0.7927 - precision: 0.8833 - recall: 0.7121 - get_f1: 0.7869
Epoch 3/10
5480/5480 [==============================] - 21s 4ms/step - loss: 0.6216 - accuracy: 0.7980 - precision: 0.8885 - recall: 0.7182 - get_f1: 0.7928
Epoch 4/10
5480/5480 [==============================] - 20s 4ms/step - loss: 2.3451 - accuracy: 0.7993 - precision: 0.8877 - recall: 0.7198 - get_f1: 0.7934
Epoch 5/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.6297 - accuracy: 0.8041 - precision: 0.8943 - recall: 0.7234 - get_f1: 0.7983
Epoch 6/10
5480/5480 [==============================] - 25s 5ms/step - loss: 0.5878 - accuracy: 0.8052 - precision: 0.8942 - recall: 0.7251 - get_f1: 0.7994
Epoch 7/10
5480/5480 [==============================] - 2

In [41]:
model = Sequential()
model.add(Conv2D(52, kernel_size=(1, 10), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 1, 30, 52)         572       
                                                                 
 flatten_7 (Flatten)         (None, 1560)              0         
                                                                 
 dense_37 (Dense)            (None, 150)               234150    
                                                                 
 dense_38 (Dense)            (None, 10)                1510      
                                                                 
Total params: 236,232
Trainable params: 236,232
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 22s 4ms/step - loss: 21.7948 - accuracy: 0.7514 - precision: 0.8534 - recall: 0.6576 - get_f1: 0.7398
Epoch 2/10
5480/5480 [==============================] - 21s 4ms/step - loss: 0.8038 - accuracy: 0.7921 - precision: 0.8833 - recall: 0.7102 - get_f1: 0.7859
Epoch 3/10
5480/5480 [==============================] - 23s 4ms/step - loss: 1.5687 - accuracy: 0.7964 - precision: 0.8872 - recall: 0.7141 - get_f1: 0.7898
Epoch 4/10
5480/5480 [==============================] - 24s 4ms/step - loss: 1.0899 - accuracy: 0.7999 - precision: 0.8875 - recall: 0.7214 - get_f1: 0.7944
Epoch 5/10
5480/5480 [==============================] - 22s 4ms/step - loss: 0.7874 - accuracy: 0.8014 - precision: 0.8897 - recall: 0.7247 - get_f1: 0.7974
Epoch 6/10
5480/5480 [==============================] - 21s 4ms/step - loss: 6.9357 - accuracy: 0.7931 - precision: 0.8810 - recall: 0.7173 - get_f1: 0.7894
Epoch 7/10
5480/5480 [==============================] - 2

In [43]:
model = Sequential()
model.add(Conv2D(52, kernel_size=(1, 2), activation='relu', input_shape=(1, 39, 1)))
model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 1, 38, 52)         156       
                                                                 
 flatten_8 (Flatten)         (None, 1976)              0         
                                                                 
 dense_39 (Dense)            (None, 150)               296550    
                                                                 
 dense_40 (Dense)            (None, 10)                1510      
                                                                 
Total params: 298,216
Trainable params: 298,216
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(data, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 31s 6ms/step - loss: 11.7137 - accuracy: 0.7627 - precision: 0.8537 - recall: 0.6825 - get_f1: 0.7564
Epoch 2/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5492 - accuracy: 0.7913 - precision: 0.8815 - recall: 0.7107 - get_f1: 0.7853
Epoch 3/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5255 - accuracy: 0.7971 - precision: 0.8881 - recall: 0.7143 - get_f1: 0.7902
Epoch 4/10
5480/5480 [==============================] - 31s 6ms/step - loss: 0.5069 - accuracy: 0.8013 - precision: 0.8916 - recall: 0.7184 - get_f1: 0.7942
Epoch 5/10
5480/5480 [==============================] - 30s 5ms/step - loss: 0.5087 - accuracy: 0.8036 - precision: 0.8929 - recall: 0.7216 - get_f1: 0.7967
Epoch 6/10
5480/5480 [==============================] - 30s 6ms/step - loss: 0.4930 - accuracy: 0.8048 - precision: 0.8950 - recall: 0.7223 - get_f1: 0.7979
Epoch 7/10
5480/5480 [==============================] - 3

In [45]:
model = Sequential()
model.add(Dense(150, input_dim=39, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 150)               6000      
                                                                 
 dense_42 (Dense)            (None, 10)                1510      
                                                                 
Total params: 7,510
Trainable params: 7,510
Non-trainable params: 0
_________________________________________________________________


In [46]:
# training the model
model.fit(input_numerical_values, output1, epochs=10)

Epoch 1/10
5480/5480 [==============================] - 7s 1ms/step - loss: 96.1783 - accuracy: 0.7302 - precision: 0.8602 - recall: 0.6102 - get_f1: 0.7106
Epoch 2/10
5480/5480 [==============================] - 6s 1ms/step - loss: 50.6764 - accuracy: 0.7665 - precision: 0.8672 - recall: 0.6715 - get_f1: 0.7552
Epoch 3/10
5480/5480 [==============================] - 6s 1ms/step - loss: 42.0881 - accuracy: 0.7713 - precision: 0.8612 - recall: 0.6835 - get_f1: 0.7605
Epoch 4/10
5480/5480 [==============================] - 6s 1ms/step - loss: 29.2569 - accuracy: 0.7793 - precision: 0.8739 - recall: 0.6909 - get_f1: 0.7702
Epoch 5/10
5480/5480 [==============================] - 6s 1ms/step - loss: 15.3408 - accuracy: 0.7836 - precision: 0.8773 - recall: 0.6958 - get_f1: 0.7746
Epoch 6/10
5480/5480 [==============================] - 6s 1ms/step - loss: 7.1027 - accuracy: 0.7886 - precision: 0.8819 - recall: 0.7024 - get_f1: 0.7804
Epoch 7/10
5480/5480 [==============================] - 6s 

In [47]:
from keras.layers import Input, Dense, Attention, dot
from keras.models import Model

data = input_numerical_values.values

query_data = data[:,:19].reshape(input_numerical_values.shape[0], 1, 19)
key_data = data[:,19:28].reshape(input_numerical_values.shape[0], 1, 9)
value_data = data[:,28:].reshape(input_numerical_values.shape[0], 1, 11)

query_input = Input(shape=(1, 19))
key_input = Input(shape=(1, 9))
value_input = Input(shape=(1, 11))

query = Dense(128, activation='relu')(query_input)
key = Dense(128, activation='relu')(key_input)
value = Dense(128, activation='relu')(value_input)

attention = Attention(10)([query, key, value])
outputs = Dense(10, activation='softmax')(attention)
model = Model(inputs=[query_input, key_input, value_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','Precision','Recall',get_f1])